In [3]:
import pandas as pd

# Load the first few rows of each dataset to inspect their structure
df_subnational = pd.read_csv('OxCGRT_compact_subnational_v1.csv')
df_national = pd.read_csv('OxCGRT_compact_national_v1.csv')

# Display the first few rows of each dataset
df_subnational.head(), df_national.head()

# Harmonizing column names (where necessary) and addressing mixed data types
# For the purpose of this analysis, we focus on columns that are essential for the hypotheses tests.

# Selecting relevant columns for the analysis, focusing on policies, cases, deaths, vaccination, and indices
columns_of_interest = [
    'CountryName', 'CountryCode', 'Date', 
    'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex_Average', 
    'GovernmentResponseIndex_Average', 'ContainmentHealthIndex_Average', 
    'EconomicSupportIndex', 'MajorityVaccinated', 'PopulationVaccinated'
]

# Creating a function to convert columns to the correct dtype and handle missing values
def prepare_dataframe(df, columns_of_interest):
    # Filter the dataframe to include only the columns of interest
    df_filtered = df[columns_of_interest].copy()
    
    # Convert 'Date' column to datetime format
    df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], format='%Y%m%d')
    
    # Convert 'MajorityVaccinated' from object to category (and handle 'NV' as NaN)
    df_filtered['MajorityVaccinated'] = df_filtered['MajorityVaccinated'].replace('NV', pd.NA).astype('category')
    
    # Fill numeric NaNs with 0 for vaccination and indices (assuming no data means no vaccination or index measure)
    numeric_columns = ['ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex_Average', 
                       'GovernmentResponseIndex_Average', 'ContainmentHealthIndex_Average', 
                       'EconomicSupportIndex', 'PopulationVaccinated']
    df_filtered[numeric_columns] = df_filtered[numeric_columns].fillna(0)
    
    return df_filtered

# Prepare both datasets
df_subnational_prepared = prepare_dataframe(df_subnational, columns_of_interest)
df_national_prepared = prepare_dataframe(df_national, columns_of_interest)

# Combining the datasets by concatenation (since we're keeping only national level from subnational, direct concat is suitable)
# Note: Subnational details were ignored in aggregation as we're matching national-level granularity for simplicity
combined_df = pd.concat([df_subnational_prepared, df_national_prepared], ignore_index=True)

# Remove any potential duplicates that might have arisen from the combination
combined_df = combined_df.drop_duplicates(subset=['CountryName', 'CountryCode', 'Date'])

combined_df.head()

/var/folders/4g/vvgynjbd1113_p3wdnd530280000gn/T/ipykernel_76004/2840183564.py:4: DtypeWarning: Columns (4,5,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df_subnational = pd.read_csv('OxCGRT_compact_subnational_v1.csv')


,CountryName,CountryCode,Date,ConfirmedCases,ConfirmedDeaths,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated
0,Australia,AUS,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
1,Australia,AUS,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2,Australia,AUS,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
3,Australia,AUS,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4,Australia,AUS,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0


In [4]:
combined_df.head()

,CountryName,CountryCode,Date,ConfirmedCases,ConfirmedDeaths,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated
0,Australia,AUS,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
1,Australia,AUS,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2,Australia,AUS,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
3,Australia,AUS,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4,Australia,AUS,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0


In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202819 entries, 0 to 442420
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   CountryName                      202819 non-null  object        
 1   CountryCode                      202819 non-null  object        
 2   Date                             202819 non-null  datetime64[ns]
 3   ConfirmedCases                   202819 non-null  float64       
 4   ConfirmedDeaths                  202819 non-null  float64       
 5   StringencyIndex_Average          202819 non-null  float64       
 6   GovernmentResponseIndex_Average  202819 non-null  float64       
 7   ContainmentHealthIndex_Average   202819 non-null  float64       
 8   EconomicSupportIndex             202819 non-null  float64       
 9   MajorityVaccinated               47406 non-null   category      
 10  PopulationVaccinated             202819 non-null 

In [8]:
combined_df.isnull().sum()

CountryName                             0
CountryCode                             0
Date                                    0
ConfirmedCases                          0
ConfirmedDeaths                         0
StringencyIndex_Average                 0
GovernmentResponseIndex_Average         0
ContainmentHealthIndex_Average          0
EconomicSupportIndex                    0
MajorityVaccinated                 155413
PopulationVaccinated                    0
dtype: int64

In [18]:
combined_df["MajorityVaccinated"].value_counts()

MajorityVaccinated
V    47406
Name: count, dtype: int64

In [20]:
combined_df=combined_df.drop(columns=["MajorityVaccinated"])

In [22]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202819 entries, 0 to 442420
Data columns (total 10 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   CountryName                      202819 non-null  object        
 1   CountryCode                      202819 non-null  object        
 2   Date                             202819 non-null  datetime64[ns]
 3   ConfirmedCases                   202819 non-null  float64       
 4   ConfirmedDeaths                  202819 non-null  float64       
 5   StringencyIndex_Average          202819 non-null  float64       
 6   GovernmentResponseIndex_Average  202819 non-null  float64       
 7   ContainmentHealthIndex_Average   202819 non-null  float64       
 8   EconomicSupportIndex             202819 non-null  float64       
 9   PopulationVaccinated             202819 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(3)
memor

In [24]:
path = "//Users/eslinkiran/Apps/cis project/combined_df_3_18"
combined_df.to_csv(path,index=False)